In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv")

In [3]:
import pandas as pd

unique_pizza_ids = df['pizza_name_id'].unique()

dataframes = {}

for pizza_id in unique_pizza_ids:
    df_filtered = df[df['pizza_name_id'] == pizza_id][['week_number', 'quantity', 'order_date']]
    dataframes[pizza_id] = df_filtered

example_pizza_id = unique_pizza_ids[0]
df = dataframes[example_pizza_id]
df

,week_number,quantity,order_date
0,1,1,2015-01-01
178,1,1,2015-01-02
356,1,1,2015-01-03
388,1,1,2015-01-03
567,1,1,2015-01-04
...,...,...,...
48056,52,1,2015-12-26
48112,52,1,2015-12-27
48211,53,1,2015-12-28
48238,53,1,2015-12-28


In [6]:
df['Order_date'] = pd.to_datetime(df['order_date'])
df

,week_number,quantity,order_date,Order_date
0,1,1,2015-01-01,2015-01-01
178,1,1,2015-01-02,2015-01-02
356,1,1,2015-01-03,2015-01-03
388,1,1,2015-01-03,2015-01-03
567,1,1,2015-01-04,2015-01-04
...,...,...,...,...
48056,52,1,2015-12-26,2015-12-26
48112,52,1,2015-12-27,2015-12-27
48211,53,1,2015-12-28,2015-12-28
48238,53,1,2015-12-28,2015-12-28


In [7]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

# Assuming your data is in a DataFrame called df
df['order_date'] = pd.to_datetime(df['order_date'])
df['week'] = df['order_date'].dt.isocalendar().week
weekly_sales = df.groupby('week')['quantity'].sum().reset_index()
weekly_sales.columns = ['week', 'quantity']

# Define and fit the SARIMA model
model = SARIMAX(weekly_sales['quantity'], 
                order=(1, 1, 1), 
                seasonal_order=(1, 1, 1, 52), 
                enforce_stationarity=False, 
                enforce_invertibility=False)
results = model.fit(disp=False)

# Forecast the next week
forecast = results.get_forecast(steps=1)
predicted_mean = forecast.predicted_mean
print(predicted_mean)

# Evaluate the model if you have a test set
test = weekly_sales.iloc[-5:]
predictions = results.get_forecast(steps=5).predicted_mean
actuals = test['quantity'].values
mape = mean_absolute_percentage_error(actuals, predictions)
print(f'MAPE: {mape}')


c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


53    3.0
dtype: float64
MAPE: 0.9096153846153845
